In [1]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'

In [1]:
import logging
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as psf

In [2]:
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", 3000) \
        .getOrCreate()

In [3]:
# kafkaStream = KafkaUtils.createStream(ssc, 'localhost:2181', "spark-streaming-consumer", {'twitter':1})

data = spark.read.json('tweets.json')

In [4]:
data.printSchema()

|    |    |    |    |    |-- source_user_id: long (nullable = true)
 |    |    |    |    |    |-- source_user_id_str: string (nullable = true)
 |    |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |    |-- video_info: struct (nullable = true)
 |    |    |    |    |    |    |-- aspect_ratio: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |    |    |    |-- duration_millis: long (nullable = true)
 |    |    |    |    |    |    |-- variants: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |-- bitrate: long (nullable = true)
 |    |    |    |    |    |    |    |    |-- content_type: string (nullable = true)
 |    |    |    |    |    |    |    |    |-- url: string (nullable = true)
 |    |    |-- extended_tweet: struct (nullable = true)
 |    

In [13]:
# parsed = kafkaStream.map(lambda v: json.loads(v[1]))

data.columns

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'limit',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user',
 'withheld_in_countries']

In [24]:
data.select('created_at', 'favorite_count', 'id', 'quote_count','reply_count', 'retweet_count', 'text').show(5)

+--------------------+--------------+-------------------+-----------+-----------+-------------+--------------------+
|          created_at|favorite_count|                 id|quote_count|reply_count|retweet_count|                text|
+--------------------+--------------+-------------------+-----------+-----------+-------------+--------------------+
|Tue Aug 18 12:12:...|             0|1295694979017453568|          0|          0|            0|j'aime trop l'équ...|
|Tue Aug 18 12:12:...|             0|1295694978967117825|          0|          0|            0|RT @ellemdiaba: e...|
|Tue Aug 18 12:12:...|             0|1295694978895589380|          0|          0|            0|RT @MiriamMLex: I...|
|Tue Aug 18 12:12:...|             0|1295694979008827392|          0|          0|            0|@shristiwtf u a c...|
|Tue Aug 18 12:12:...|             0|1295694978962915328|          0|          0|            0| I’m good love enjoy|
+--------------------+--------------+-------------------+-------

In [31]:
data.createOrReplaceTempView('tweets')

df = spark.sql("select id, quote_count, reply_count, retweet_count, text, lang from tweets where lang='en'")
df.show(10)

+-------------------+-----------+-----------+-------------+--------------------+----+
|                 id|quote_count|reply_count|retweet_count|                text|lang|
+-------------------+-----------+-----------+-------------+--------------------+----+
|1295694978895589380|          0|          0|            0|RT @MiriamMLex: I...|  en|
|1295694979008827392|          0|          0|            0|@shristiwtf u a c...|  en|
|1295694978962915328|          0|          0|            0| I’m good love enjoy|  en|
|1295694979038240768|          0|          0|            0|wiped 31% CECDDDA...|  en|
|1295694977633320960|          0|          0|            0|Uko na sooo zicom...|  en|
|1295694978757341185|          0|          0|            0|RT @TrumpWarRoom:...|  en|
|1295694978920972289|          0|          0|            0|      Good luck 🙌🏾|  en|
|1295694979013124096|          0|          0|            0|RT @rymrtlorbes: ...|  en|
|1295694979046821891|          0|          0|           

In [ ]:
ssc.start()
ssc.awaitTermination(timeout=180)